In [1]:
%matplotlib inline
#from keras.applications.resnet50 import ResNet50
# not using keras version since it cannot have an input size < 197x197.  Might also require 3-channel color.
import keras
from keras.optimizers import RMSprop
from keras.utils import np_utils
import resnet
from glob import glob
import numpy as np
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import json

Using TensorFlow backend.
/home/rallen/anaconda3/envs/dl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#K.image_data_format()
print(keras.backend.image_data_format())
print(keras.backend.image_dim_ordering())

channels_last
tf


In [3]:
train_path = 'data/train3'
train_input_image_names = glob(train_path+'/crop_score*png')
train_input_image_names.sort()
train_input_images = np.stack([np.array(Image.open(fn)) for fn in train_input_image_names])
train_input_images.shape = train_input_images.shape + (1,)
num_train_images,rows,cols,channels = train_input_images.shape
num_train_images,rows,cols,channels

(2553, 120, 32, 1)

In [4]:
valid_path = 'data/valid3'
valid_input_image_names = glob(valid_path+'/crop_score*png')
valid_input_image_names.sort()
valid_input_images = np.stack([np.array(Image.open(fn)) for fn in valid_input_image_names])
valid_input_images.shape = valid_input_images.shape + (1,)
num_valid_images,rows,cols,channels = valid_input_images.shape
num_valid_images,rows,cols,channels

(851, 120, 32, 1)

In [5]:
results_json = 'data/train3/crop_score_info.json'
with open(results_json,'r') as f:
    results = json.load(f)

In [6]:
train_indices = []
for train_name in train_input_image_names:
    index = int(train_name.split('.')[0].split('_')[-1])
    train_indices.append(index)
valid_indices = []
for valid_name in valid_input_image_names:
    index = int(valid_name.split('.')[0].split('_')[-1])
    valid_indices.append(index)
#train_indices[:20],valid_indices[:20]

In [7]:
notes = np.zeros(128+1,dtype=np.int32)
lengths = np.zeros(16+1,dtype=np.int32)
for ti in train_indices:
    r = results[ti] # [84, 4] = 
    notes[r[0]] += 1
    lengths[r[1]] += 1

In [8]:
# kind of sparse, could be optimized
notes,lengths

(array([1921,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    1,    2,    1,   23,   31,   26,   34,   33,   20,
          35,   33,   34,   22,   40,   21,   17,   32,   21,   33,   16,
          15,   28,   18,   30,   16,   32,   16,    2,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], dtype=int32),
 array([1921,   46,  192,    0,  240,    0,    0,    0,   84,    0,    0,
           0,    0,    0,    0,    0,   7

In [9]:
# HMM, realizing that MULTIPLE classes of output (note + length) are not 
# easily mapped here...going to need to look at this harder.  For now
# let's just try to see if we can do notes
#np.concatenate([lengths,notes])
#NUM_CATEGORIES=128+1 + 16+1
NUM_CATEGORIES=128+1 
#train_results = np.zeros([len(train_indices),NUM_CATEGORIES],dtype=np.float32)
train_results = np.zeros(len(train_indices),dtype=np.float32)
i = 0
for ti in train_indices:
    r = results[ti]
    #train_results[i,16+1+r[0]] = 1.0 # set note value
    #train_results[i,r[1]] = 1.0 # set duration value
    train_results[i] = r[0]
    i += 1
train_results = np_utils.to_categorical(train_results,NUM_CATEGORIES)

In [10]:
np_utils.to_categorical([0,1,2,1,0],3)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.]])

In [11]:
train_results.shape

(2553, 129)

In [12]:
#valid_results = np.zeros([len(valid_indices),128+1 + 16+1],dtype=np.float32)
valid_results = np.zeros(len(valid_indices),dtype=np.float32)
i = 0
for ti in valid_indices:
    r = results[ti]
    #valid_results[i,16+1+r[0]] = 1.0 # set note value
    #valid_results[i,r[1]] = 1.0 # set duration value
    valid_results[i] = r[0]
    i += 1
valid_results = np_utils.to_categorical(valid_results,NUM_CATEGORIES)

In [13]:
train_results[100]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [14]:
class score_generator(object):
    """X should be list of score images, Y should be a list of score results.  
    Both should be the same length."""
    def __init__(self, X, Y, batch_size, channels):
        self.X  = X
        self.Y  = Y
        self.bs = batch_size
        self.channels = channels
        self.i  = 0
    def __next__(self):
        xs = self.X[self.i:self.i+self.bs]
        ys = self.Y[self.i:self.i+self.bs]
        #ys = ys.reshape(len(ys),-1,1)
        self.i = (self.i + self.bs) % self.X.shape[0]
        return xs, ys

In [15]:
batch_size = 32
train_generator = score_generator(train_input_images, train_results, batch_size, channels)
valid_generator = score_generator(valid_input_images, valid_results, batch_size, channels)

In [16]:
#model = resnet.ResnetBuilder.build_resnet_18((64, 64, 1), 128) # Total params: 11,444,288
#model = resnet.ResnetBuilder.build_resnet_34((64, 64, 1), 128) # Total params: 21,563,584
#model = resnet.ResnetBuilder.build_resnet_50((64, 64, 1), 128) # Total params: 24,025,920
image_shape = (rows,cols,channels)
#output_bins = 128+1 + 16+1 # midi values + note length
output_bins = NUM_CATEGORIES
#model = resnet.ResnetBuilder.build_resnet_50(image_shape, output_bins)
model = resnet.ResnetBuilder.build_resnet_18(image_shape, output_bins)
# Total params: 24,234,320
# Trainable params: 24,188,880
# Non-trainable params: 45,440

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 32, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 16, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 60, 16, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 60, 16, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [18]:
train_rate  = 1e-3
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(train_rate),
              metrics=["accuracy"])

In [19]:
num_train_images//batch_size, num_valid_images//batch_size

(79, 26)

In [22]:
num_epochs = 50
model.fit_generator(train_generator,
                    num_train_images//batch_size, num_epochs,
                    verbose=2,
                    validation_data=valid_generator,
                    validation_steps=num_valid_images//batch_size)

Epoch 1/50
 - 3s - loss: 0.4030 - acc: 0.9763 - val_loss: 0.4686 - val_acc: 0.9540
Epoch 2/50
 - 3s - loss: 0.3357 - acc: 0.9830 - val_loss: 0.9685 - val_acc: 0.8471
Epoch 3/50
 - 3s - loss: 0.3023 - acc: 0.9865 - val_loss: 0.4952 - val_acc: 0.9434
Epoch 4/50
 - 3s - loss: 0.2818 - acc: 0.9897 - val_loss: 0.3120 - val_acc: 0.9792
Epoch 5/50
 - 3s - loss: 0.2955 - acc: 0.9824 - val_loss: 0.3930 - val_acc: 0.9550
Epoch 6/50
 - 3s - loss: 0.2405 - acc: 0.9929 - val_loss: 0.3140 - val_acc: 0.9691
Epoch 7/50
 - 3s - loss: 0.2153 - acc: 0.9909 - val_loss: 0.2678 - val_acc: 0.9795
Epoch 8/50
 - 3s - loss: 0.1960 - acc: 0.9905 - val_loss: 0.3818 - val_acc: 0.9664
Epoch 9/50
 - 3s - loss: 0.1866 - acc: 0.9897 - val_loss: 0.3464 - val_acc: 0.9550
Epoch 10/50
 - 3s - loss: 0.2103 - acc: 0.9897 - val_loss: 0.3913 - val_acc: 0.9444
Epoch 11/50
 - 3s - loss: 0.2133 - acc: 0.9877 - val_loss: 0.2485 - val_acc: 0.9828
Epoch 12/50
 - 3s - loss: 0.1613 - acc: 0.9941 - val_loss: 0.2027 - val_acc: 0.9850
E

In [21]:
x = next(train_generator)
x[0].shape

(32, 120, 32, 1)

In [23]:
test_path = 'data/test3'
test_input_image_names = glob(test_path+'/crop_score*png')
test_input_image_names.sort()
test_input_images = np.stack([np.array(Image.open(fn)) for fn in test_input_image_names])
test_input_images.shape = test_input_images.shape + (1,)
num_test_images,rows,cols,channels = test_input_images.shape
num_test_images,rows,cols,channels

(851, 120, 32, 1)

In [25]:
test_indices = []
for test_name in test_input_image_names:
    index = int(test_name.split('.')[0].split('_')[-1])
    test_indices.append(index)

In [26]:
test_results = np.zeros(len(test_indices),dtype=np.float32)
i = 0
for ti in test_indices:
    r = results[ti]
    test_results[i] = r[0]
    i += 1
test_results = np_utils.to_categorical(test_results,NUM_CATEGORIES)

In [28]:
test_generator = score_generator(test_input_images, test_results, batch_size, channels)

In [29]:
model.evaluate_generator(test_generator,steps=num_test_images//batch_size)

[0.16321852215780661, 0.97836538461538458]

In [30]:
# not bad!